# **Homework 6 - Generative Adversarial Network**

This is the example code of homework 6 of the machine learning course by Prof. Hung-yi Lee.


In this homework, you are required to build a generative adversarial  network for anime face generation.


# Reference

In [ ]:
# https://github.com/NVlabs/stylegan2-ada-pytorch
# @inproceedings{Karras2020ada,
#   title     = {Training Generative Adversarial Networks with Limited Data},
#   author    = {Tero Karras and Miika Aittala and Janne Hellsten and Samuli Laine and Jaakko Lehtinen and Timo Aila},
#   booktitle = {Proc. NeurIPS},
#   year      = {2020}
# }

# 利用stylegan2的data_tool 對資料做處理後，利用其模型架構（train.py）對模型做訓練，最後利用generate.py產生輸出圖片。

SyntaxError: ignored

## Set up the environment


### Packages Installation

In [1]:
import torch
try:
    # Get GPU name, check if it's K80
    GPU_name = torch.cuda.get_device_name()
    if GPU_name[-3:] == "K80":
        print("Get K80! :'( RESTART!")
    else:
        print("Your GPU is {}!".format(GPU_name))
        print("Great! Keep going~")
except RuntimeError as e:
    if e.args == ("No CUDA GPUs are available",):
        print("You are training with CPU! "
              "Please restart!")
    else:
        print("What's wrong here?")
        print("Error message: \n", e)

Your GPU is Tesla V100-SXM2-16GB!
Great! Keep going~


In [2]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 125, done.
remote: Total 125 (delta 0), reused 0 (delta 0), pack-reused 125
Receiving objects: 100% (125/125), 1.12 MiB | 4.61 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [3]:
# You may replace the workspace directory if you want.
# workspace_dir = '/content/drive/MyDrive/ML/hw6'
workspace_dir = './'

# Training progress bar
!pip install -q qqdm

### Download the dataset
**Please use the link according to the last digit of your student ID first!**

If all download links fail, please follow [here](https://drive.google.com/drive/folders/13T0Pa_WGgQxNkqZk781qhc5T9-zfh19e).

* To open the file using your browser, use the link below (replace the id first!):
https://drive.google.com/file/d/REPLACE_WITH_ID
* e.g. https://drive.google.com/file/d/1IGrTr308mGAaCKotpkkm8wTKlWs9Jq-p

In [ ]:
# !gdown --id 1IGrTr308mGAaCKotpkkm8wTKlWs9Jq-p --output "{workspace_dir}/crypko_data.zip"

# Other download links
#   Please uncomment the line according to the last digit of your student ID first

# 0
# !gdown --id 131zPaVoi-U--XThvzgRfaxrumc3YSBd3 --output "{workspace_dir}/crypko_data.zip"

# 1
# !gdown --id 1kCuIj1Pf3T2O94H9bUBxjPBKb---WOmH --output "{workspace_dir}/crypko_data.zip"

# 2
# !gdown --id 1boEoiiqBJwoHVvjmI0xgoutE9G0Rv8CD --output "{workspace_dir}/crypko_data.zip"

# 3
# !gdown --id 1Ic0mktAQQvnNAnswrPHsg-u2OWGBXTWF --output "{workspace_dir}/crypko_data.zip"

# 4
# !gdown --id 1PFcc25r9tLE7OyQ-CDadtysNdWizk6Yg --output "{workspace_dir}/crypko_data.zip"

# 5
# !gdown --id 1wgkrYkTrhwDSMdWa5NwpXeE4-7JaUuX2 --output "{workspace_dir}/crypko_data.zip"

# 6
# !gdown --id 19gwNYWi9gN9xVL86jC3v8qqNtrXyq5Bf --output "{workspace_dir}/crypko_data.zip"

# 7 
# !gdown --id 1-KPZB6frRSRLRAtQfafKCVA7em0_NrJG --output "{workspace_dir}/crypko_data.zip"

# 8
# !gdown --id 1rNBfmn0YBzXuG5ub7CXbsGwduZqEs8hx --output "{workspace_dir}/crypko_data.zip"

# 9
# !gdown --id 113NEISX-2j6rBd1yyBx0c3_9nPIzSNz- --output "{workspace_dir}/crypko_data.zip"


Downloading...
From: https://drive.google.com/uc?id=1boEoiiqBJwoHVvjmI0xgoutE9G0Rv8CD
To: /content/crypko_data.zip
452MB [00:12, 37.3MB/s]


###Unzip the downloaded file.
The unzipped tree structure is like 
```
faces/
├── 1.jpg
├── 2.jpg
├── 3.jpg
...
```

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp /content/drive/MyDrive/ML/hw6/crypko_data.zip ./crypko_data.zip

In [ ]:
# !unzip -q "{workspace_dir}/crypko_data.zip" -d "{workspace_dir}/"

## Random seed
Set the random seed to a certain value for reproducibility.

In [6]:
import random

import torch
import numpy as np


def same_seeds(seed):
    # Python built-in random module
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Torch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(2021)

## Import Packages
First, we need to import packages that will be used later.

Like hw3, we highly rely on **torchvision**, a library of PyTorch.

In [7]:
import os
import glob

import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch import optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from qqdm.notebook import qqdm

## Dataset
1. Resize the images to (64, 64)
1. Linearly map the values from [0, 1] to  [-1, 1].

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms.


In [ ]:
# reference在最上方
!python ./stylegan2-ada-pytorch/dataset_tool.py --source=/content/crypko_data.zip --dest=./data.zip --width=64 --height=64

100% 71314/71314 [01:34<00:00, 752.19it/s]


In [ ]:
# class CrypkoDataset(Dataset):
#     def __init__(self, fnames, transform):
#         self.transform = transform
#         self.fnames = fnames
#         self.num_samples = len(self.fnames)

#     def __getitem__(self,idx):
#         fname = self.fnames[idx]
#         # 1. Load the image
#         img = torchvision.io.read_image(fname)
#         # 2. Resize and normalize the images using torchvision.
#         img = self.transform(img)
#         return img

#     def __len__(self):
#         return self.num_samples


# def get_dataset(root):
#     fnames = glob.glob(os.path.join(root, '*'))
#     # 1. Resize the image to (64, 64)
#     # 2. Linearly map [0, 1] to [-1, 1]
#     compose = [
#         transforms.ToPILImage(),
#         transforms.Resize((64, 64)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
#     ]
#     transform = transforms.Compose(compose)
#     dataset = CrypkoDataset(fnames, transform)
#     return dataset

### Show some images
Note that the values are in the range of [-1, 1], we should shift them to the valid range, [0, 1], to display correctly.

In [ ]:
# images = [(dataset[i]+1)/2 for i in range(16)]
# grid_img = torchvision.utils.make_grid(images, nrow=4)
# plt.figure(figsize=(10,10))
# plt.imshow(grid_img.permute(1, 2, 0))
# plt.show()

## Model
Here, we use DCGAN as the model structure. Feel free to modify your own model structure.

Note that the `N` of the input/output shape stands for the batch size.

In [ ]:
# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Conv') != -1:
#         m.weight.data.normal_(0.0, 0.02)
#     elif classname.find('BatchNorm') != -1:
#         m.weight.data.normal_(1.0, 0.02)
#         m.bias.data.fill_(0)

# class Generator(nn.Module):
#     """
#     Input shape: (N, in_dim)
#     Output shape: (N, 3, 64, 64)
#     """
#     def __init__(self, in_dim, dim=64):
#         super(Generator, self).__init__()
#         def dconv_bn_relu(in_dim, out_dim):
#             return nn.Sequential(
#                 nn.ConvTranspose2d(in_dim, out_dim, 5, 2,
#                                    padding=2, output_padding=1, bias=False),
#                 nn.BatchNorm2d(out_dim),
#                 nn.ReLU()
#             )
#         self.l1 = nn.Sequential(
#             nn.Linear(in_dim, dim * 8 * 4 * 4, bias=False),
#             nn.BatchNorm1d(dim * 8 * 4 * 4),
#             nn.ReLU()
#         )
#         self.l2_5 = nn.Sequential(
#             dconv_bn_relu(dim * 8, dim * 4),
#             dconv_bn_relu(dim * 4, dim * 2),
#             dconv_bn_relu(dim * 2, dim),
#             nn.ConvTranspose2d(dim, 3, 5, 2, padding=2, output_padding=1),
#             nn.Tanh()
#         )
#         self.apply(weights_init)

#     def forward(self, x):
#         y = self.l1(x)
#         y = y.view(y.size(0), -1, 4, 4)
#         y = self.l2_5(y)
#         return y


# class Discriminator(nn.Module):
#     """
#     Input shape: (N, 3, 64, 64)
#     Output shape: (N, )
#     """
#     def __init__(self, in_dim, dim=64):
#         super(Discriminator, self).__init__()

#         def conv_bn_lrelu(in_dim, out_dim):
#             return nn.Sequential(
#                 nn.Conv2d(in_dim, out_dim, 5, 2, 2),
#                 nn.BatchNorm2d(out_dim),
#                 nn.LeakyReLU(0.2),
#             )
            
#         """ Medium: Remove the last sigmoid layer for WGAN. """
#         self.ls = nn.Sequential(
#             nn.Conv2d(in_dim, dim, 5, 2, 2), 
#             nn.LeakyReLU(0.2),
#             conv_bn_lrelu(dim, dim * 2),
#             conv_bn_lrelu(dim * 2, dim * 4),
#             conv_bn_lrelu(dim * 4, dim * 8),
#             nn.Conv2d(dim * 8, 1, 4),
#             # nn.Sigmoid(), 
#         )
#         self.apply(weights_init)
        
#     def forward(self, x):
#         y = self.ls(x)
#         y = y.view(-1)
#         return y

## Training

### Initialization
- hyperparameters
- model
- optimizer
- dataloader

In [ ]:
# # Training hyperparameters
# batch_size = 64
# z_dim = 100
# z_sample = Variable(torch.randn(100, z_dim)).cuda()
# lr = 1e-4

# """ Medium: WGAN, 50 epoch, n_critic=5, clip_value=0.01 """
# n_epoch = 50 # 50
# n_critic = 5 # 5
# clip_value = 0.01

# log_dir = os.path.join(workspace_dir, 'logs')
# ckpt_dir = os.path.join(workspace_dir, 'checkpoints')
# os.makedirs(log_dir, exist_ok=True)
# os.makedirs(ckpt_dir, exist_ok=True)

# # Model
# G = Generator(in_dim=z_dim).cuda()
# D = Discriminator(3).cuda()
# G.train()
# D.train()

# # Loss
# criterion = nn.BCELoss()

# """ Medium: Use RMSprop for WGAN. """
# # Optimizer
# # opt_D = torch.optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))
# # opt_G = torch.optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
# opt_D = torch.optim.RMSprop(D.parameters(), lr=0.0004)
# opt_G = torch.optim.RMSprop(G.parameters(), lr=0.0001)


# # DataLoader
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)


NameError: ignored

### Training loop
We store some pictures regularly to monitor the current performance of the Generator, and regularly record checkpoints.

In [ ]:
# steps = 0
# for e, epoch in enumerate(range(n_epoch)):
#     progress_bar = qqdm(dataloader)
#     for i, data in enumerate(progress_bar):
#         imgs = data
#         imgs = imgs.cuda()

#         bs = imgs.size(0)

#         # ============================================
#         #  Train D
#         # ============================================
#         z = Variable(torch.randn(bs, z_dim)).cuda()
#         r_imgs = Variable(imgs).cuda()
#         f_imgs = G(z)

#         """ Medium: Use WGAN Loss. """
#         # # Label
#         # r_label = torch.ones((bs)).cuda()
#         # f_label = torch.zeros((bs)).cuda()

#         # # Model forwarding
#         # r_logit = D(r_imgs.detach())
#         # f_logit = D(f_imgs.detach())
        
#         # # Compute the loss for the discriminator.
#         # r_loss = criterion(r_logit, r_label)
#         # f_loss = criterion(f_logit, f_label)
#         # loss_D = (r_loss + f_loss) / 2

#         # WGAN Loss
#         loss_D = -torch.mean(D(r_imgs)) + torch.mean(D(f_imgs))
       

#         # Model backwarding
#         D.zero_grad()
#         loss_D.backward()

#         # Update the discriminator.
#         opt_D.step()

#         """ Medium: Clip weights of discriminator. """
#         for p in D.parameters():
#            p.data.clamp_(-clip_value, clip_value)

#         # ============================================
#         #  Train G
#         # ============================================
#         if steps % n_critic == 0:
#             # Generate some fake images.
#             z = Variable(torch.randn(bs, z_dim)).cuda()
#             f_imgs = G(z)

#             # Model forwarding
#             f_logit = D(f_imgs)
            
#             """ Medium: Use WGAN Loss"""
#             # # Compute the loss for the generator.
#             # loss_G = criterion(f_logit, r_label)
#             # WGAN Loss
#             loss_G = -torch.mean(D(f_imgs))

#             # Model backwarding
#             G.zero_grad()
#             loss_G.backward()

#             # Update the generator.
#             opt_G.step()

#         steps += 1
        
#         # Set the info of the progress bar
#         #   Note that the value of the GAN loss is not directly related to
#         #   the quality of the generated images.
#         progress_bar.set_infos({
#             'Loss_D': round(loss_D.item(), 4),
#             'Loss_G': round(loss_G.item(), 4),
#             'Epoch': e+1,
#             'Step': steps,
#         })

#     G.eval()
#     f_imgs_sample = (G(z_sample).data + 1) / 2.0
#     filename = os.path.join(log_dir, f'Epoch_{epoch+1:03d}.jpg')
#     torchvision.utils.save_image(f_imgs_sample, filename, nrow=10)
#     print(f' | Save some samples to {filename}.')
    
#     # Show generated images in the jupyter notebook.
#     grid_img = torchvision.utils.make_grid(f_imgs_sample.cpu(), nrow=10)
#     plt.figure(figsize=(10,10))
#     plt.imshow(grid_img.permute(1, 2, 0))
#     plt.show()
#     G.train()

#     if (e+1) % 5 == 0 or e == 0:
#         # Save the checkpoints.
#         torch.save(G.state_dict(), os.path.join(ckpt_dir, 'G.pth'))
#         torch.save(D.state_dict(), os.path.join(ckpt_dir, 'D.pth'))



In [8]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

     |████████████████████████████████| 204kB 4.3MB/s 
     |████████████████████████████████| 112kB 44.4MB/s 
     |████████████████████████████████| 26.9MB 65.4MB/s 


In [ ]:
# reference在最上方
!python /content/stylegan2-ada-pytorch/train.py \
--data=./data.zip --batch=64 --outdir="/content/drive/MyDrive/ML/hw6/boss" \
--cfg=stylegan2 --seed=2021 --snap=30 --kimg=1000 --gpus=1


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 30,
  "network_snapshot_ticks": 30,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 2021,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "./data.zip",
    "use_labels": false,
    "max_size": 71314,
    "xflip": false,
    "resolution": 64
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512,
    "num

## Inference
Use the trained model to generate anime faces!

### Load model 

In [22]:
# import torch_utils

# G = Generator(z_dim)
# G.load_state_dict(torch.load("/content/drive/MyDrive/ML/hw6/boss2/00000-data-stylegan2-kimg1000-batch64/network-snapshot-001000.pkl"))
# G.eval()
# G.cuda()

### Generate and show some images.


In [19]:
# reference在最上方
!rm -r /content/output
%cd /content/stylegan2-ada-pytorch
!python generate.py --outdir=/content/output --trunc=0.4 --seeds=1-1000 \
--network=/content/drive/MyDrive/ML/hw6/boss2/00000-data-stylegan2-kimg1000-batch64/network-snapshot-000604.pkl

/content/stylegan2-ada-pytorch
Loading networks from "/content/drive/MyDrive/ML/hw6/boss2/00000-data-stylegan2-kimg1000-batch64/network-snapshot-000241.pkl"...
Generating image for seed 1 (0/1000) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 2 (1/1000) ...
Generating image for seed 3 (2/1000) ...
Generating image for seed 4 (3/1000) ...
Generating image for seed 5 (4/1000) ...
Generating image for seed 6 (5/1000) ...
Generating image for seed 7 (6/1000) ...
Generating image for seed 8 (7/1000) ...
Generating image for seed 9 (8/1000) ...
Generating image for seed 10 (9/1000) ...
Generating image for seed 11 (10/1000) ...
Generating image for seed 12 (11/1000) ...
Generating image for seed 13 (12/1000) ...
Generating image for seed 14 (13/1000) ...
Generating image for seed 15 (14/1000) ...
Generating image for seed 16 (15/1000) ...
Generating image for seed 17 (16/1000) ...
Generating image for 

### Compress the generated images using **tar**.


In [20]:
# 轉png為jpg
%cd /content/output
from PIL import Image
from os import listdir
from os.path import splitext

target_directory = '.'
target = '.jpg'

for file in listdir(target_directory):
    filename, extension = splitext(file)
    try:
        if extension not in ['.py', target]:
            im = Image.open(filename + extension)
            im.save(filename[4:] + target, "JPEG")
    except OSError:
        print('Cannot convert %s' % file)

!rm *.png

/content/output


In [21]:
# # Save the generated images.
# os.makedirs('/content/drive/MyDrive/ML/hw6/boss2/00000-data-stylegan2-kimg1000-batch64/output', exist_ok=True)
# for i in range(1000):
#     torchvision.utils.save_image(img, f'/content/drive/MyDrive/ML/hw6/boss2/00000-data-stylegan2-kimg1000-batch64/output/{i+1}.jpg')
  
# Compress the images.
%cd /content/output
!tar -zcf ../images.tgz *.jpg
%cd ..

/content/output
/content
